## Schwarzschild horizon in 3+1 Eddington-Finkelstein coordinates

In [1]:
%display latex

In [2]:
M = Manifold(4, 'M')
print M

4-dimensional differentiable manifold M


In [3]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

In [4]:
var('m', domain='real')
assume(m>=0)

In [5]:
g = M.lorentzian_metric('g')
g[0,0] = -(1-2*m/r)
g[0,1] = 2*m/r
g[1,1] = 1+2*m/r
g[2,2] = r^2
g[3,3] = (r*sin(th))^2
g.display()

g = (2*m - r)/r dt*dt + 2*m/r dt*dr + 2*m/r dr*dt + (2*m + r)/r dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

In [6]:
g[:]

[  (2*m - r)/r         2*m/r             0             0]
[        2*m/r   (2*m + r)/r             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

Let us check that we have a solution of Einstein equation in vacuum: 

In [7]:
g.ricci().display()

Ric(g) = 0

In [8]:
# l = X.frame()[0].copy()
l = M.vector_field()
l[0] = 1
l[1] = (r-2*m)/(r+2*m)
l.set_name('l', latex_name=r'\ell')
print l
l.display()

Vector field l on the 4-dimensional differentiable manifold M


l = d/dt - (2*m - r)/(2*m + r) d/dr

In [9]:
g(l,l).expr()

0

In [31]:
l_form = l.down(g)
l_form.set_name('lf', latex_name=r'\underline{\ell}')
print l_form
l_form.display()

1-form lf on the 4-dimensional differentiable manifold M


lf = (2*m - r)/(2*m + r) dt + dr

In [11]:
nab = g.connection()
print nab
nab

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional differentiable manifold M


Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional differentiable manifold M

In [12]:
nab_l_form = nab(l_form)
nab_l_form.display()

nabla_g(ul) = (2*m^2 - m*r)/(2*m*r^2 + r^3) dt*dt + (4*m^3 + 4*m^2*r - 3*m*r^2)/(4*m^2*r^2 + 4*m*r^3 + r^4) dt*dr + m/r^2 dr*dt + (2*m^2 + 3*m*r)/(2*m*r^2 + r^3) dr*dr - (2*m*r - r^2)/(2*m + r) dth*dth - (2*m*r - r^2)*sin(th)^2/(2*m + r) dph*dph

In [13]:
nab_l_form.symmetrize().display()

(2*m^2 - m*r)/(2*m*r^2 + r^3) dt*dt + (4*m^3 + 4*m^2*r - m*r^2)/(4*m^2*r^2 + 4*m*r^3 + r^4) dt*dr + (4*m^3 + 4*m^2*r - m*r^2)/(4*m^2*r^2 + 4*m*r^3 + r^4) dr*dt + (2*m^2 + 3*m*r)/(2*m*r^2 + r^3) dr*dr - (2*m*r - r^2)/(2*m + r) dth*dth - (2*m*r - r^2)*sin(th)^2/(2*m + r) dph*dph

In [14]:
v = l.contract(nab_l_form, 0)
v.display()

0

In [15]:
nab_l = nab(l)
nab_l[:]

[                                                   m/r^2                          (2*m^2 + 3*m*r)/(2*m*r^2 + r^3)                                                        0                                                        0]
[                          -(2*m^2 - m*r)/(2*m*r^2 + r^3) -(4*m^3 + 4*m^2*r - 3*m*r^2)/(4*m^2*r^2 + 4*m*r^3 + r^4)                                                        0                                                        0]
[                                                       0                                                        0                                 -(2*m - r)/(2*m*r + r^2)                                                        0]
[                                                       0                                                        0                                                        0                                 -(2*m - r)/(2*m*r + r^2)]

In [16]:
div_l = nab_l.trace()
print div_l
div_l.display()

Scalar field on the 4-dimensional differentiable manifold M


M --> R
(t, r, th, ph) |--> -2*(4*m^2 - 2*m*r - r^2)/(4*m^2*r + 4*m*r^2 + r^3)

In [17]:
div_l.expr().subs(r=2*m)

1/4/m

In [18]:
acc_l = l.contract(0,nab_l,1)
print acc_l
acc_l.display()

Vector field on the 4-dimensional differentiable manifold M


4*m/(4*m^2 + 4*m*r + r^2) d/dt - 4*(2*m^2 - m*r)/(8*m^3 + 12*m^2*r + 6*m*r^2 + r^3) d/dr

In [19]:
kappa = acc_l[[0]]
acc_l == kappa * l

True

In [20]:
kappa0 = kappa.expr().subs(r=2*m)
kappa0

1/4/m

In [21]:
k = M.vector_field(name='k')
k[0] = 1/2 + m/r
k[1] = -1/2 - m/r
k.display()

k = 1/2*(2*m + r)/r d/dt - 1/2*(2*m + r)/r d/dr

In [22]:
g(k,k).expr()

0

In [23]:
g(k,l).expr()

-1

In [24]:
k_form = k.down(g)
k_form.set_name('kf', latex_name=r'\underline{k}')
k_form.display()

kf = -1/2*(2*m + r)/r dt - 1/2*(2*m + r)/r dr

In [25]:
q = g + l_form*k_form + k_form*l_form
q.set_name('q')
q.display()

q = r^2 dth*dth + r^2*sin(th)^2 dph*dph

In [26]:
q[0,0].expr().subs(r=2*m)

0

In [27]:
q[0,1].expr().subs(r=2*m)

0

In [28]:
q[1,1].expr().subs(r=2*m)

0

In [29]:
q_up = q.up(g)
print q_up
q_up.display()

Tensor field of type (2,0) on the 4-dimensional differentiable manifold M


r^(-2) d/dth*d/dth + 1/(r^2*sin(th)^2) d/dph*d/dph

In [30]:
theta = q_up.contract(0,1,nab(l_form),0,1)
theta.expr()

-2*(2*m - r)/(2*m*r + r^2)

In [32]:
theta == div_l - kappa

True